In [1]:
import numpy as np

class ANN:
    def __init__(self,input_size,hidden_layers,hidden_neurons,output_size,learning_rate):
        self.weights=[]
        self.bias=[]
        self.hidden_layers=hidden_layers
        self.learning_rate=learning_rate;
        
        for i in range(hidden_layers+1):
            if i==0:
                self.weights.append(np.random.randn(hidden_neurons,input_size))
                self.bias.append(np.full((hidden_neurons,1),1))
            elif i==hidden_layers:
                self.weights.append(np.random.randn(output_size,hidden_neurons))
                self.bias.append(np.full((output_size,1),1))
            else:
                self.weights.append(np.random.randn(hidden_neurons,hidden_neurons))
                self.bias.append(np.full((hidden_neurons,1),1))
                
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def first_order_sigmoid(self, x):
        return self.sigmoid(x) * (1 - self.sigmoid(x))
            
    def forward(self,x):
        activations=[]
        activations.append(x)
        for i in range(self.hidden_layers+1):
            x=np.dot(self.weights[i],activations[i])+self.bias[i]
            activations.append(self.sigmoid(x))
        return activations

    def backward(self,activations,di,m):
        delta=(activations[-1]-di.T) * self.first_order_sigmoid(np.dot(self.weights[-1],activations[-2])+self.bias[-1])
        for i in range(self.hidden_layers,-1,-1):
            if i==self.hidden_layers:
                prev=np.array(self.weights[i])
                self.weights[i]=self.weights[i]-(self.learning_rate/m) * np.dot(delta,activations[i].T)
                self.bias[i]=self.bias[i]-(self.learning_rate/m) * np.sum(delta,axis=1,keepdims=True)
            else:
                delta=np.dot(prev.T, delta) * self.first_order_sigmoid(np.dot(self.weights[i], activations[i])+self.bias[i])
                prev=np.array(self.weights[i])
                self.weights[i]=self.weights[i]-(self.learning_rate/m) * np.dot(delta,activations[i].T)
                self.bias[i]=self.bias[i]-(self.learning_rate/m) * np.sum(delta,axis=1,keepdims=True)
            
    def train(self,x,y,epochs):
        for i in range(epochs):
            activations=self.forward(x)
            m=x.shape[1]
            self.backward(activations,y,m)
            if(i%1000==0):
                print("Error at %d epoch : "%(i),np.sum(activations[-1]-y.T))
            
    def predict(self,x):
        predictions=[]
        for input in x:
            prediction = self.forward(np.array(input))
            predictions.append(prediction[-1])
        return predictions
        

In [2]:
input_size=4
hidden_layers=2
neurons_in_hidden_layer=5
output_size=3
learning_rate=0.1

model=ANN(input_size,hidden_layers,neurons_in_hidden_layer,output_size,learning_rate)

In [3]:
import pandas as pd
import seaborn as sns
# df=pd.read_csv('Iris.csv')
df=sns.load_dataset('iris')

# x = df.drop(['Id','Species'], axis=1)
x = df.drop(['species'], axis=1)
y=df['species']

In [4]:
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [5]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(y)

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,encoded_labels,test_size=0.2,random_state=62)

from tensorflow.keras.utils import to_categorical

labels = np.array(y_train)
y_train = to_categorical(labels)
y_train=np.array(y_train)

In [6]:
# x_train=x_train.values
# x_train=np.array(x_train)
# x_train=x_train.T
# x_test=x_test.values

x_train = x_train.to_numpy().T
x_test = x_test.to_numpy()


In [7]:
print('x_train \n',x_train)
print('x_test\n',x_test)
print('y_train\n',y_train)

x_train 
 [[5.5 5.9 5.8 6.9 5.8 4.6 6.4 5.8 5.9 6.3 6.1 6.9 5.4 4.6 6.7 5.9 5.  7.9
  5.  6.5 5.1 5.8 4.3 6.1 5.4 7.7 5.5 5.5 4.4 5.  4.8 5.  7.  4.7 6.5 6.7
  5.6 5.7 5.7 5.7 4.9 7.1 5.8 5.4 6.7 5.7 4.6 6.4 5.5 6.  6.1 5.7 5.2 5.6
  4.4 6.2 4.6 6.2 6.8 5.8 5.6 5.  6.8 5.4 5.6 5.5 4.4 5.  5.2 5.1 6.3 6.6
  6.4 5.1 5.6 5.4 6.3 7.7 7.2 6.3 4.9 6.  7.7 6.9 6.1 5.4 6.5 5.8 5.  6.9
  5.6 6.1 6.3 5.7 4.9 6.7 7.7 6.5 4.8 7.2 6.4 6.  4.9 5.1 4.8 6.  5.1 6.7
  6.1 5.  6.2 6.3 5.1 6.6 4.8 6.4 6.4 5.5 4.8 5.7]
 [2.3 3.2 2.7 3.2 4.  3.2 2.7 2.7 3.  2.5 3.  3.1 3.  3.4 2.5 3.  3.4 3.8
  3.  3.  3.8 2.7 3.  2.9 3.4 3.8 3.5 2.6 3.  3.5 3.  2.  3.2 3.2 3.2 3.1
  3.  2.6 2.5 2.8 3.  3.  2.6 3.7 3.3 3.  3.1 2.9 2.5 3.4 2.6 4.4 3.4 3.
  3.2 2.2 3.6 2.8 3.2 2.7 2.9 3.3 3.  3.4 2.5 2.4 2.9 3.6 2.7 3.8 2.8 3.
  3.2 3.3 2.8 3.9 3.4 2.6 3.2 2.3 3.1 3.  3.  3.1 3.  3.9 3.  2.8 3.2 3.1
  2.7 2.8 2.5 2.8 2.4 3.1 2.8 3.  3.  3.  2.8 2.9 2.5 3.7 3.1 2.7 3.5 3.
  2.8 2.3 2.9 3.3 3.5 2.9 3.4 3.2 3.1 2.4 3.4 3.8]
 [4

In [8]:
epochs=100000
model.train(x_train,y_train,epochs)

Error at 0 epoch :  149.64532491356545
Error at 1000 epoch :  7.109550986258574
Error at 2000 epoch :  10.884159310169743
Error at 3000 epoch :  7.864596518916681
Error at 4000 epoch :  7.580949153571813
Error at 5000 epoch :  6.926865161707125
Error at 6000 epoch :  6.01042352261116
Error at 7000 epoch :  4.729504358629377
Error at 8000 epoch :  3.8177350728551267
Error at 9000 epoch :  3.2394791533529173
Error at 10000 epoch :  2.8543602930802456
Error at 11000 epoch :  2.581098423489478
Error at 12000 epoch :  2.3763315264664593
Error at 13000 epoch :  2.2161048164995423
Error at 14000 epoch :  2.0864676513796345
Error at 15000 epoch :  1.9788654772473944
Error at 16000 epoch :  1.8877949595654253
Error at 17000 epoch :  1.8095318212641185
Error at 18000 epoch :  1.7414104911529944
Error at 19000 epoch :  1.6814292606559074
Error at 20000 epoch :  1.6280485135239386
Error at 21000 epoch :  1.5800807632294762
Error at 22000 epoch :  1.5366128044474516
Error at 23000 epoch :  1.496940

In [9]:
hh=model.forward([[6.7],[3.0],[5.2],[2.3]])
print(hh[-1])

[[5.56101364e-05]
 [7.73355567e-03]
 [9.96960820e-01]]


In [10]:
test_sample=[]
for i in x_test:
    test_sample.append([[x] for x in i.tolist()])

In [11]:
# y_pred=model.predict(test_sample)
# y_pred = np.hstack([np.argmax(arr, axis=0) for arr in y_pred]).flatten()

y_pred = np.argmax(np.hstack(model.predict(test_sample)), axis=0)

In [12]:
print(y_test)

[2 2 0 0 2 1 2 0 1 1 2 2 0 1 1 1 0 1 0 0 0 0 2 1 2 0 2 0 2 0]


In [13]:
print(y_pred)

[2 2 0 0 2 1 2 0 1 1 2 2 0 1 1 1 0 1 0 0 0 0 2 1 2 0 2 0 2 0]


In [14]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,confusion_matrix
accuracy=accuracy_score(y_test,y_pred)
precision=precision_score(y_test,y_pred,average='weighted')
recall=recall_score(y_test,y_pred,average='weighted')
f1score=f1_score(y_test,y_pred,average='weighted')
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1score)

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-score: 1.0
